In [3]:
# !wget https://huggingface.co/datasets/malaysia-ai/Malaysian-NSFW/resolve/main/malaysian-dataset-sfw-dedup-v2.jsonl

In [16]:
import json
from tqdm import tqdm
import requests
import os

In [5]:
texts = []
with open('malaysian-dataset-sfw-dedup-v2.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        texts.append(l['text'])

In [9]:
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}

json_data = {
    'input': 'string',
}

response = requests.post('http://api/v1/embeddings', headers=headers, json=json_data)

In [15]:
!mkdir nsfw-embedding

In [21]:
for i in tqdm(range(len(texts))):
    filename = os.path.join('nsfw-embedding', f'{i}.json')
    if os.path.exists(filename):
        continue
    
    json_data = {
        'input': texts[i],
    }

    response = requests.post('http://100.93.25.29:7077/v1/embeddings', headers=headers, json=json_data)
    
    with open(filename, 'w') as fopen:
        json.dump(response.json(), fopen)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 203177/203177 [1:49:34<00:00, 30.90it/s]


In [22]:
texts = []
with open('malaysian-dataset-sfw-dedup-v2.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        texts.append(l)

In [23]:
texts[0]

{'text': 'I feel like a useless friend, I promised myself that I will support my friends as best as I can. But I also hope that my friends understand my capabilities.',
 'label': 'psychiatric or mental illness'}

In [32]:
with open('nsfw-embedding.jsonl', 'w') as fopen_l:
    for i in tqdm(range(len(texts))):
        filename = os.path.join('nsfw-embedding', f'{i}.json')
        if not os.path.exists(filename):
            continue
        with open(filename) as fopen:
            d = json.load(fopen)
        d = {
            'embedding': d['data'][0]['embedding'],
            **texts[i]
        }
        fopen_l.write(f'{json.dumps(d)}\n')
        fopen_l.flush()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 203177/203177 [03:03<00:00, 1106.24it/s]


In [ ]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj="nsfw-embedding.jsonl",
    path_in_repo="nsfw-embedding.jsonl",
    repo_id="malaysia-ai/Malaysian-NSFW-embedding",
    repo_type="dataset",
)

nsfw-embedding.jsonl:   0%|          | 0.00/3.60G [00:00<?, ?B/s]